<h3>Implementierung von Principal Coponent Pursuit </h3>

<p>Den Algorithmus findet ihr hier unter "https://github.com/dfm/pcp". Es handelt sich um eine der Methoden zur Berechnung der PCA, welche auch im Paper unter Punkt 5.2 (Comparison) zu finden sind. Hierbei wird die PCA auf ein Minimierungsproblem zurückgeführt, welches mit Hilfe einer Singulärwertzerlegung gelöst wird. Weitere Infos zu diesem Verfahren unter "http://arxiv.org/abs/0912.3599".</p>



In [1]:
"""
An implementation of the Principal Component Pursuit algorithm for robust PCA
as described in `Candes, Li, Ma, & Wright <http://arxiv.org/abs/0912.3599>`_.
An alternative Python implementation using non-standard dependencies and
different hyperparameter choices is available at:
http://blog.shriphani.com/2013/12/18/
    robust-principal-component-pursuit-background-matrix-recovery/
"""

from __future__ import division, print_function

__all__ = ["pcp"]

import time
import fbpca
import logging
import numpy as np
from scipy.sparse.linalg import svds


def pcp(M, delta=1e-6, mu=None, maxiter=500, verbose=False, missing_data=True,
        svd_method="approximate", **svd_args):
    # Check the SVD method.
    allowed_methods = ["approximate", "exact", "sparse"]
    if svd_method not in allowed_methods:
        raise ValueError("'svd_method' must be one of: {0}"
                         .format(allowed_methods))

    # Check for missing data.
    shape = M.shape
    if missing_data:
        missing = ~(np.isfinite(M))
        if np.any(missing):
            M = np.array(M)
            M[missing] = 0.0
    else:
        missing = np.zeros_like(M, dtype=bool)
        if not np.all(np.isfinite(M)):
            logging.warn("The matrix has non-finite entries. "
                         "SVD will probably fail.")

    # Initialize the tuning parameters.
    lam = 1.0 / np.sqrt(np.max(shape))
    if mu is None:
        mu = 0.25 * np.prod(shape) / np.sum(np.abs(M))
        if verbose:
            print("mu = {0}".format(mu))

    # Convergence criterion.
    norm = np.sum(M ** 2)

    # Iterate.
    i = 0
    rank = np.min(shape)
    S = np.zeros(shape)
    Y = np.zeros(shape)
    while i < max(maxiter, 1):
        # SVD step.
        strt = time.time()
        u, s, v = _svd(svd_method, M - S + Y / mu, rank+1, 1./mu, **svd_args)
        svd_time = time.time() - strt

        s = shrink(s, 1./mu)
        rank = np.sum(s > 0.0)
        u, s, v = u[:, :rank], s[:rank], v[:rank, :]
        L = np.dot(u, np.dot(np.diag(s), v))

        # Shrinkage step.
        S = shrink(M - L + Y / mu, lam / mu)

        # Lagrange step.
        step = M - L - S
        step[missing] = 0.0
        Y += mu * step

        # Check for convergence.
        err = np.sqrt(np.sum(step ** 2) / norm)
        if verbose:
            print(("Iteration {0}: error={1:.3e}, rank={2:d}, nnz={3:d}, "
                   "time={4:.3e}")
                  .format(i, err, np.sum(s > 0), np.sum(S > 0), svd_time))
        if err < delta:
            break
        i += 1

    if i >= maxiter:
        logging.warn("convergence not reached in pcp")
    return L, S, (u, s, v)


def shrink(M, tau):
    sgn = np.sign(M)
    S = np.abs(M) - tau
    S[S < 0.0] = 0.0
    return sgn * S


def _svd(method, X, rank, tol, **args):
    rank = min(rank, np.min(X.shape))
    if method == "approximate":
        return fbpca.pca(X, k=rank, raw=True, **args)
    elif method == "exact":
        return np.linalg.svd(X, full_matrices=False, **args)
    elif method == "sparse":
        if rank >= np.min(X.shape):
            return np.linalg.svd(X, full_matrices=False)
        u, s, v = svds(X, k=rank, tol=tol)
        u, s, v = u[:, ::-1], s[::-1], v[::-1, :]
        return u, s, v
    raise ValueError("invalid SVD method")
    


<h3>Import der Kursdaten aus den letzten 6 Monaten zu 5 Aktien aus dem Dax 30 </h3>

In [2]:
import numpy as np
import csv

with open('../Stock prices dax 30.csv') as stockprices:
    data = list(csv.reader(stockprices, delimiter=";"))

In [3]:
data_array = np.array(data)
data_only = data_array[1:data_array.shape[0],1:6].T
data_only = np.array([[float(y) for y in x] for x in data_only])
data_only.shape

(5, 128)

<h3>Berechnung der (empirischen) Kovarianzmatrix basierend auf den gegebene Kursdaten </h3>

In [4]:
Sigma = np.zeros((5,5))

for k in range(0,data_only.shape[1] - 1):
    
    Sigma = Sigma + np.dot(np.subtract(data_only[:,k], np.mean(data_only, axis = 1)).reshape((5,1)), np.subtract(data_only[:,k], np.mean(data_only, axis = 1)).reshape((1,5)))

Sigma = (1/(data_only.shape[1] - 1)) * Sigma

Sigma

array([[142.67041515,  28.06338926,  30.56147946,   2.52487121,
         31.18558268],
       [ 28.06338926,  14.54671933, -10.75409144,  -2.97700557,
         20.0735403 ],
       [ 30.56147946, -10.75409144,  65.84451884,  11.30075662,
        -28.04105723],
       [  2.52487121,  -2.97700557,  11.30075662,  10.38498412,
         -5.84017695],
       [ 31.18558268,  20.0735403 , -28.04105723,  -5.84017695,
         33.39091805]])

<h3>Anwendung von PCP zur Durchführung der robusten Hauptkomponentenanalyse (PCA) </h3>

In [27]:
L, S = pcp(Sigma)[0:2]

In [28]:
L

array([[ 33.15288862,  19.03429426,  -2.38353047,   2.5249416 ,
         24.04138395],
       [ 19.03429426,  14.54671744, -10.7540876 ,  -2.97707117,
         20.07354269],
       [ -2.38353047, -10.7540876 ,  24.51612531,  11.30069593,
        -17.99311351],
       [  2.5249416 ,  -2.97707117,  11.30069593,   5.60790277,
         -5.84023382],
       [ 24.04138395,  20.07354269, -17.99311351,  -5.84023382,
         28.30038404]])

In [29]:
S

array([[109.51752654,   9.029095  ,  32.94500994,   0.        ,
          7.14419873],
       [  9.029095  ,   0.        ,  -0.        ,  -0.        ,
         -0.        ],
       [ 32.94500994,  -0.        ,  41.32839353,   0.        ,
        -10.04794373],
       [  0.        ,  -0.        ,   0.        ,   4.77708135,
          0.        ],
       [  7.14419873,  -0.        , -10.04794373,   0.        ,
          5.09053401]])

In [30]:
L + S

array([[142.67041515,  28.06338926,  30.56147946,   2.5249416 ,
         31.18558268],
       [ 28.06338926,  14.54671744, -10.7540876 ,  -2.97707117,
         20.07354269],
       [ 30.56147946, -10.7540876 ,  65.84451884,  11.30069593,
        -28.04105723],
       [  2.5249416 ,  -2.97707117,  11.30069593,  10.38498412,
         -5.84023382],
       [ 31.18558268,  20.07354269, -28.04105723,  -5.84023382,
         33.39091805]])

In [33]:
from numpy.linalg import matrix_rank
matrix_rank(L)

2